# Nhận diện khuôn mặt thời gian thực với FaceNet & MTCNN trên Webcam

Bài tập thực hành: Xây dựng hệ thống nhận diện khuôn mặt theo thời gian thực sử dụng:
- **MTCNN**: Phát hiện khuôn mặt
- **FaceNet**: Trích xuất đặc trưng và so sánh khuôn mặt
- **OpenCV**: Truy cập webcam

**Điều kiện so sánh:**
- Similarity > 0.7: "Matched"
- Similarity < 0.7: "Unknown"

## 1. Import thư viện và thiết lập cấu hình

Phần này import các thư viện cần thiết và thiết lập các tham số quan trọng:
- **MTCNN**: Thư viện phát hiện khuôn mặt với độ chính xác cao
- **FaceNet**: Mô hình deep learning để trích xuất đặc trưng khuôn mặt (embeddings 512 chiều)
- **OpenCV**: Xử lý ảnh và video

**Các tham số cấu hình:**
- `KNOWN_DIR`: Thư mục chứa ảnh của những người cần nhận diện
- `THRESHOLD`: Ngưỡng similarity để xác định "Matched" (0.7 = 70% độ tương đồng)
- `CAM_INDEX`: Chỉ số webcam (0 = webcam mặc định)
- `MIN_FACE_SIZE`: Kích thước tối thiểu của khuôn mặt được xử lý (pixel)

In [ ]:
import os
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet

# =========================
# CONFIG
# =========================
KNOWN_DIR = "known"          # thư mục chứa ảnh người đã biết
THRESHOLD = 0.7             # similarity threshold
CAM_INDEX = 0               # webcam index (0 thường là webcam mặc định)
MIN_FACE_SIZE = 60          # bỏ qua mặt quá nhỏ (px)

print("✅ Đã import các thư viện và thiết lập config")

## 2. Định nghĩa các hàm hỗ trợ (Helper Functions)

Các hàm tiện ích để xử lý embeddings và ảnh:

- **`l2_normalize()`**: Chuẩn hóa vector theo chuẩn L2 (đưa về độ dài = 1)
- **`cosine_similarity()`**: Tính độ tương đồng cosine giữa 2 embeddings (kết quả từ -1 đến 1, càng gần 1 = càng giống)
- **`preprocess_face()`**: Tiền xử lý khuôn mặt:
  - Chuyển từ BGR → RGB (OpenCV dùng BGR, FaceNet cần RGB)
  - Resize về 160×160 pixels (kích thước input của FaceNet)
- **`get_embedding()`**: Trích xuất vector đặc trưng 512 chiều từ ảnh khuôn mặt
- **`safe_crop()`**: Crop ảnh an toàn không bị vượt biên frame

In [ ]:
# =========================
# HELPERS
# =========================
def l2_normalize(x: np.ndarray, eps: float = 1e-10) -> np.ndarray:
    """Chuẩn hóa vector theo L2"""
    return x / (np.linalg.norm(x) + eps)

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    """Tính độ tương đồng cosine giữa 2 vectors"""
    a = l2_normalize(a)
    b = l2_normalize(b)
    return float(np.dot(a, b))

def preprocess_face(face_bgr: np.ndarray, target_size=(160, 160)) -> np.ndarray:
    """
    FaceNet thường dùng input 160x160, RGB.
    Trả về mảng (160,160,3) RGB uint8.
    """
    face_rgb = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB)
    face_rgb = cv2.resize(face_rgb, target_size, interpolation=cv2.INTER_AREA)
    return face_rgb

def get_embedding(embedder: FaceNet, face_rgb_160: np.ndarray) -> np.ndarray:
    """
    keras-facenet: embeddings() nhận list/np array ảnh RGB (uint8 ok).
    """
    emb = embedder.embeddings([face_rgb_160])[0]  # shape (512,)
    return emb.astype(np.float32)

def safe_crop(frame: np.ndarray, x: int, y: int, w: int, h: int) -> tuple:
    """Crop an toàn, đảm bảo không vượt biên"""
    H, W = frame.shape[:2]
    x1 = max(0, x)
    y1 = max(0, y)
    x2 = min(W, x + w)
    y2 = min(H, y + h)
    return frame[y1:y2, x1:x2], (x1, y1, x2, y2)

print("✅ Đã định nghĩa các helper functions")

## 3. Khởi tạo các mô hình AI

Load 2 mô hình deep learning chính:

1. **MTCNN (Multi-task Cascaded Convolutional Networks)**:
   - Phát hiện khuôn mặt trong ảnh/video
   - Trả về bounding box và confidence score
   - Hoạt động tốt với nhiều khuôn mặt, góc nghiêng, và điều kiện ánh sáng khác nhau

2. **FaceNet (keras-facenet)**:
   - Mô hình CNN pre-trained trên hàng triệu khuôn mặt
   - Chuyển đổi ảnh khuôn mặt thành vector 512 chiều
   - Các khuôn mặt giống nhau có embeddings gần nhau trong không gian vector

In [ ]:
# =========================
# LOAD MODELS
# =========================
print("Đang load models...")
detector = MTCNN()
embedder = FaceNet()  # load FaceNet
print("✅ MTCNN và FaceNet đã sẵn sàng!")

## 4. Xây dựng cơ sở dữ liệu khuôn mặt (Database)

Tạo database embeddings từ ảnh trong thư mục `known/`:

**Quy trình:**
1. Tạo thư mục `known/` nếu chưa tồn tại
2. Duyệt qua tất cả file ảnh (.jpg, .jpeg, .png)
3. Với mỗi ảnh:
   - Đọc ảnh và phát hiện khuôn mặt bằng MTCNN
   - Lấy khuôn mặt có confidence cao nhất (nếu có nhiều người)
   - Crop và preprocess khuôn mặt
   - Trích xuất embedding 512 chiều bằng FaceNet
   - Lưu embedding và tên người (tên file không đuôi) vào arrays
4. Chuyển sang numpy array để tính toán hiệu quả

**Kết quả:** Danh sách `known_embeddings` và `known_names` để so sánh real-time

In [ ]:
# =========================
# BUILD KNOWN DATABASE
# =========================
known_embeddings = []
known_names = []

# Tạo thư mục nếu chưa có
if not os.path.isdir(KNOWN_DIR):
    os.makedirs(KNOWN_DIR, exist_ok=True)
    print(f"[!] Đã tạo thư mục '{KNOWN_DIR}'. Hãy bỏ ảnh người quen vào đó rồi chạy lại.")
else:
    image_files = [f for f in os.listdir(KNOWN_DIR) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    
    if len(image_files) == 0:
        print(f"[!] Thư mục '{KNOWN_DIR}' chưa có ảnh. Hãy thêm ảnh (jpg/png) rồi chạy lại.")
    else:
        print(f"[*] Đang tạo database embeddings từ thư mục {KNOWN_DIR}/ ...")
        
        for fn in image_files:
            path = os.path.join(KNOWN_DIR, fn)
            img = cv2.imread(path)
            if img is None:
                print(f"[!] Không đọc được: {path}")
                continue

            # detect face in known image
            faces = detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if len(faces) == 0:
                print(f"[!] Không phát hiện mặt trong: {fn}")
                continue

            # lấy face có confidence cao nhất
            faces = sorted(faces, key=lambda d: d.get("confidence", 0), reverse=True)
            x, y, w, h = faces[0]["box"]
            face_crop, _ = safe_crop(img, x, y, w, h)

            if face_crop.size == 0 or min(face_crop.shape[:2]) < MIN_FACE_SIZE:
                print(f"[!] Mặt quá nhỏ/lỗi crop trong: {fn}")
                continue

            face_160 = preprocess_face(face_crop)
            emb = get_embedding(embedder, face_160)

            name = os.path.splitext(fn)[0]  # tên = filename không đuôi
            known_embeddings.append(emb)
            known_names.append(name)
            print(f"    + Loaded: {name}")

        known_embeddings = np.array(known_embeddings, dtype=np.float32)
        
        if len(known_embeddings) == 0:
            print("[!] Không tạo được embeddings nào từ known/. Hãy dùng ảnh rõ mặt hơn.")
        else:
            print(f"\n✅ Database sẵn sàng: {len(known_embeddings)} người")
            print(f"📋 Danh sách: {', '.join(known_names)}")

## 5. Nhận diện khuôn mặt theo thời gian thực (Real-time Recognition)

Xử lý video từ webcam và nhận diện khuôn mặt frame-by-frame:

**Luồng xử lý mỗi frame:**
1. Capture frame từ webcam và flip ngang (hiệu ứng gương)
2. Chuyển sang RGB và phát hiện khuôn mặt bằng MTCNN
3. Với mỗi khuôn mặt được phát hiện:
   - Kiểm tra confidence > 0.90 (lọc detection kém)
   - Crop và preprocess khuôn mặt
   - Trích xuất embedding
   - **So sánh với database**: Tính cosine similarity với tất cả known embeddings
   - Lấy similarity cao nhất và kiểm tra threshold:
     - **> 0.7**: Matched → vẽ box XANH LÁ + hiển thị tên
     - **≤ 0.7**: Unknown → vẽ box ĐỎ
4. Hiển thị frame với bounding boxes, tên, và similarity scores
5. Nhấn 'q' để thoát

**Tối ưu:**
- Bỏ qua khuôn mặt quá nhỏ (< 60px)
- Hiển thị confidence của MTCNN
- Đếm số frames đã xử lý

**Việc chỉ xử lý các khuôn mặt có confidence > 0.90 nhằm:**

Loại bỏ các phát hiện kém chính xác do nhiễu nền, ánh sáng yếu hoặc vật thể giống khuôn mặt.

Giảm sai số khi trích xuất embedding: nếu vùng crop không phải khuôn mặt thật, FaceNet sẽ sinh vector đặc trưng sai lệch, dẫn đến nhận diện nhầm.

Tăng độ ổn định của hệ thống realtime, tránh việc nhãn “Matched/Unknown” nhấp nháy liên tục.

Tối ưu hiệu năng, chỉ xử lý những khuôn mặt có chất lượng tốt, giảm tải tính toán không cần thiết.

Ngưỡng 0.90 được chọn dựa trên thực nghiệm, đảm bảo cân bằng giữa độ chính xác phát hiện và khả năng nhận diện ổn định trong môi trường webcam thời gian thực.

In [ ]:
# =========================
# REALTIME WEBCAM
# =========================
if len(known_embeddings) == 0:
    print("⚠️  Không có database để so sánh. Hãy thêm ảnh vào thư mục 'known/' trước!")
else:
    cap = cv2.VideoCapture(CAM_INDEX)
    if not cap.isOpened():
        print("[!] Không mở được webcam.")
    else:
        print("[*] Webcam đã sẵn sàng. Nhấn 'q' để thoát.")
        print(f"[*] Threshold: {THRESHOLD}")
        print(f"[*] Số người trong database: {len(known_names)}")
        print("-" * 50)
        
        frame_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Flip frame để tạo hiệu ứng gương
            frame = cv2.flip(frame, 1)
            
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            detections = detector.detect_faces(rgb)

            for det in detections:
                conf = det.get("confidence", 0)
                if conf < 0.90:  # bỏ qua detection có confidence thấp
                    continue

                x, y, w, h = det["box"]
                face_crop, (x1, y1, x2, y2) = safe_crop(frame, x, y, w, h)

                if face_crop.size == 0:
                    continue
                if min(face_crop.shape[:2]) < MIN_FACE_SIZE:
                    continue

                face_160 = preprocess_face(face_crop)
                emb_live = get_embedding(embedder, face_160)

                # So khớp: lấy similarity lớn nhất
                sims = [cosine_similarity(emb_live, e) for e in known_embeddings]
                best_idx = int(np.argmax(sims))
                best_sim = float(sims[best_idx])
                best_name = known_names[best_idx]

                # Kiểm tra threshold
                if best_sim > THRESHOLD:
                    label = "Matched"
                    color = (0, 255, 0)  # XANH LÁ
                    # display_text = f"{best_name} | sim={best_sim:.2f}"  #hiển thị tên
                    display_text = f"MATCHED | sim={best_sim:.2f}"
                else:
                    label = "Unknown"
                    color = (0, 0, 255)  # ĐỎ
                    display_text = f"UNKNOWN | sim={best_sim:.2f}"

                # Vẽ bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                
                # Vẽ background cho text
                (tw, th), _ = cv2.getTextSize(display_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                cv2.rectangle(frame, (x1, y1-30), (x1+tw, y1), color, -1)
                
                # Vẽ text
                cv2.putText(frame, display_text, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # Vẽ confidence
                cv2.putText(frame, f"Conf: {conf:.2f}", (x1, y2+20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Hiển thị hướng dẫn
            cv2.putText(frame, "Press 'q' to quit", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Hiển thị frame
            cv2.imshow("FaceNet + MTCNN Realtime", frame)
            
            frame_count += 1
            
            if cv2.waitKey(25) & 0xFF == ord("q"):
                break

        cap.release()
        cv2.destroyAllWindows()
        print("\n✅ Đã đóng webcam và giải phóng tài nguyên")
        print(f"📊 Tổng số frames xử lý: {frame_count}")